## Deliverable 2. Create a Customer Travel Destinations Map.

In [11]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
from datetime import datetime
# Import the time module.
import time
# Get today's date in seconds.
today = time.time()
# Import the API key.
from config import weather_api_key
from config import g_key

In [12]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Carnarvon,AU,-24.8667,113.6333,62.67,63.00%,75.00%,13.80,broken clouds
1,1,Vagur,FO,61.4732,-6.8119,48.16,81.00%,100.00%,24.02,overcast clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,46.02,65.00%,40.00%,26.46,light rain
3,3,Bluff,NZ,-46.6000,168.3333,52.81,76.00%,24.00%,6.11,few clouds
4,4,Cape Town,ZA,-33.9258,18.4232,59.09,56.00%,0.00%,4.00,clear sky


In [9]:
#2 Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Vila Franca Do Campo,PT,37.7167,-25.4333,75.20,86.00%,100.00%,3.78,overcast clouds
8,8,Fairhope,US,30.5230,-87.9033,82.06,77.00%,40.00%,6.91,scattered clouds
12,12,Sao Paulo De Olivenca,BR,-3.3783,-68.8725,84.72,70.00%,97.00%,4.72,light rain
13,13,Kapaa,US,22.0752,-159.3190,81.19,72.00%,20.00%,3.00,light rain
15,15,Hirara,JP,24.8000,125.2833,82.89,67.00%,56.00%,19.95,broken clouds
27,27,Susanville,US,40.4163,-120.6530,79.83,12.00%,1.00%,13.00,clear sky
32,32,Lorengau,PG,-2.0226,147.2712,80.62,78.00%,100.00%,3.62,overcast clouds
34,34,Bengkulu,ID,-3.8004,102.2655,75.25,93.00%,100.00%,4.85,light rain
40,40,Cayenne,GF,4.9333,-52.3333,86.04,75.00%,20.00%,5.01,few clouds
41,41,Poya,NC,-21.3500,165.1500,78.73,63.00%,0.00%,2.62,clear sky


In [29]:
# 4a. Determine if there are any empty rows.
#preferred_cities_df.head()
preferred_cities_df.isnull().values.any()
#preferred_cities_df.isnull().sum().sum()

False

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = pd.DataFrame(preferred_cities_df)
clean_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Vila Franca Do Campo,PT,37.7167,-25.4333,75.20,86.00%,100.00%,3.78,overcast clouds
8,8,Fairhope,US,30.5230,-87.9033,82.06,77.00%,40.00%,6.91,scattered clouds
12,12,Sao Paulo De Olivenca,BR,-3.3783,-68.8725,84.72,70.00%,97.00%,4.72,light rain
13,13,Kapaa,US,22.0752,-159.3190,81.19,72.00%,20.00%,3.00,light rain
15,15,Hirara,JP,24.8000,125.2833,82.89,67.00%,56.00%,19.95,broken clouds
27,27,Susanville,US,40.4163,-120.6530,79.83,12.00%,1.00%,13.00,clear sky
32,32,Lorengau,PG,-2.0226,147.2712,80.62,78.00%,100.00%,3.62,overcast clouds
34,34,Bengkulu,ID,-3.8004,102.2655,75.25,93.00%,100.00%,4.85,light rain
40,40,Cayenne,GF,4.9333,-52.3333,86.04,75.00%,20.00%,5.01,few clouds
41,41,Poya,NC,-21.3500,165.1500,78.73,63.00%,0.00%,2.62,clear sky


In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Vila Franca Do Campo,PT,75.20,overcast clouds,37.7167,-25.4333,
8,Fairhope,US,82.06,scattered clouds,30.5230,-87.9033,
12,Sao Paulo De Olivenca,BR,84.72,light rain,-3.3783,-68.8725,
13,Kapaa,US,81.19,light rain,22.0752,-159.3190,
15,Hirara,JP,82.89,broken clouds,24.8000,125.2833,
27,Susanville,US,79.83,clear sky,40.4163,-120.6530,
32,Lorengau,PG,80.62,overcast clouds,-2.0226,147.2712,
34,Bengkulu,ID,75.25,light rain,-3.8004,102.2655,
40,Cayenne,GF,86.04,few clouds,4.9333,-52.3333,
41,Poya,NC,78.73,clear sky,-21.3500,165.1500,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...  skipping.")  

Hotel not found...  skipping.
Hotel not found...  skipping.
Hotel not found...  skipping.
Hotel not found...  skipping.
Hotel not found...  skipping.
Hotel not found...  skipping.
Hotel not found...  skipping.
Hotel not found...  skipping.
Hotel not found...  skipping.
Hotel not found...  skipping.
Hotel not found...  skipping.
Hotel not found...  skipping.
Hotel not found...  skipping.
Hotel not found...  skipping.
Hotel not found...  skipping.
Hotel not found...  skipping.
Hotel not found...  skipping.


In [31]:
hotel_df.isna().any()
#hotel_df.head()

City                   False
Country                False
Max Temp               False
Current Description    False
Lat                    False
Lng                    False
Hotel Name             False
dtype: bool

In [21]:
# 8a. Create the output File (CSV)
hotel_df.to_csv('WeatherPy_vacation.csv')

# 8b. Export the City_Data into a csv
clean_df.to_csv('City_Data.csv', index_label="City_ID")


In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>

<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=.85)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))